In [20]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
import pathlib

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

import tensorflow as tf

In [3]:
tf.__version__

'2.3.1'

In [4]:
data = pd.read_csv("data/parenting_dataset_asli.csv", index_col="NO")
data.drop(columns=["Responden"], inplace=True)
data.head()

,AE1,AE2,AE3,AE4,AE5,AE6,AE7,AE8,AE9,AE10,...,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10,Parenting_style
NO,,,,,,,,,,,,,,,,,,,,,
1,1,5,2,5,1,4,1,4,1,5,...,4,3,1,4,1,2,4,1,1,Authoration
2,1,4,2,4,2,5,1,4,5,5,...,2,3,2,3,4,3,2,2,1,Authorative
3,2,4,4,5,5,4,2,5,5,5,...,4,3,4,4,4,4,4,3,3,Authorative
4,1,3,2,4,2,4,1,3,2,5,...,5,2,4,3,3,3,5,5,4,Permissive
5,1,1,3,3,1,1,1,5,3,3,...,5,1,4,3,4,3,4,4,3,Permissive


In [5]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [6]:
X = data.drop(['Parenting_style'], axis = 1)
Y = data['Parenting_style']

print("Shape of Input  features: {}".format(X.shape))
print("Shape of Output features: {}".format(Y.shape))

Shape of Input  features: (500, 52)
Shape of Output features: (500,)


In [7]:
Y.value_counts()

Authorative    182
Authoration    163
Permissive     155
Name: Parenting_style, dtype: int64

In [8]:
X.head()

,AE1,AE2,AE3,AE4,AE5,AE6,AE7,AE8,AE9,AE10,...,PE1,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10
NO,,,,,,,,,,,,,,,,,,,,,
1,1,5,2,5,1,4,1,4,1,5,...,1,4,3,1,4,1,2,4,1,1
2,1,4,2,4,2,5,1,4,5,5,...,1,2,3,2,3,4,3,2,2,1
3,2,4,4,5,5,4,2,5,5,5,...,4,4,3,4,4,4,4,4,3,3
4,1,3,2,4,2,4,1,3,2,5,...,5,5,2,4,3,3,3,5,5,4
5,1,1,3,3,1,1,1,5,3,3,...,2,5,1,4,3,4,3,4,4,3


In [9]:
lbl_clf = LabelEncoder()
Y_encoded = lbl_clf.fit_transform(Y)

#Keras requires your output feature to be one-hot encoded values.
Y_final = tf.keras.utils.to_categorical(Y_encoded)

print("Therefore, our final shape of output feature will be {}".format(Y_final.shape))

Therefore, our final shape of output feature will be (500, 3)


In [10]:
x_train, x_test, y_train, y_test = train_test_split(X, Y_final, test_size=0.2, random_state=seed, stratify=Y_encoded, shuffle=True)

print("Training Input shape\t: {}".format(x_train.shape))
print("Testing Input shape\t: {}".format(x_test.shape))
print("Training Output shape\t: {}".format(y_train.shape))
print("Testing Output shape\t: {}".format(y_test.shape))

Training Input shape	: (400, 52)
Testing Input shape	: (100, 52)
Training Output shape	: (400, 3)
Testing Output shape	: (100, 3)


In [11]:
std_clf = StandardScaler()
x_train_new = std_clf.fit_transform(x_train)
x_test_new = std_clf.transform(x_test)

In [14]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(16, input_dim=52, activation='relu', kernel_initializer='he_normal', 
                                kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(8, activation='relu', kernel_initializer='he_normal', 
                                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(8, activation='relu', kernel_initializer='he_normal', 
                                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_history = model.fit(x_train_new, y_train, epochs=200, batch_size=5)

Epoch 1/200
80/80 [==============================] - 0s 1ms/step - loss: 1.9795 - accuracy: 0.3625
Epoch 2/200
80/80 [==============================] - 0s 1ms/step - loss: 1.6729 - accuracy: 0.3925
Epoch 3/200
80/80 [==============================] - 0s 3ms/step - loss: 1.5486 - accuracy: 0.4025
Epoch 4/200
80/80 [==============================] - 0s 988us/step - loss: 1.5483 - accuracy: 0.3800
Epoch 5/200
80/80 [==============================] - 0s 988us/step - loss: 1.4837 - accuracy: 0.4050
Epoch 6/200
80/80 [==============================] - 0s 987us/step - loss: 1.4706 - accuracy: 0.4325
Epoch 7/200
80/80 [==============================] - 0s 1ms/step - loss: 1.4318 - accuracy: 0.5025
Epoch 8/200
80/80 [==============================] - 0s 988us/step - loss: 1.4143 - accuracy: 0.4650
Epoch 9/200
80/80 [==============================] - 0s 1ms/step - loss: 1.4094 - accuracy: 0.4400
Epoch 10/200
80/80 [==============================] - 0s 988us/step - loss: 1.3666 - accuracy: 0.4950

80/80 [==============================] - 0s 1ms/step - loss: 0.6828 - accuracy: 0.7825
Epoch 83/200
80/80 [==============================] - 0s 1ms/step - loss: 0.6473 - accuracy: 0.8275
Epoch 84/200
80/80 [==============================] - 0s 1ms/step - loss: 0.6889 - accuracy: 0.7800
Epoch 85/200
80/80 [==============================] - 0s 975us/step - loss: 0.7047 - accuracy: 0.7675
Epoch 86/200
80/80 [==============================] - 0s 988us/step - loss: 0.6342 - accuracy: 0.7950
Epoch 87/200
80/80 [==============================] - 0s 975us/step - loss: 0.6297 - accuracy: 0.8100
Epoch 88/200
80/80 [==============================] - 0s 988us/step - loss: 0.5461 - accuracy: 0.8325
Epoch 89/200
80/80 [==============================] - 0s 1ms/step - loss: 0.6192 - accuracy: 0.7775
Epoch 90/200
80/80 [==============================] - 0s 1ms/step - loss: 0.6784 - accuracy: 0.7950
Epoch 91/200
80/80 [==============================] - 0s 1ms/step - loss: 0.6694 - accuracy: 0.8175
Epoch

80/80 [==============================] - 0s 1ms/step - loss: 0.4864 - accuracy: 0.8575
Epoch 163/200
80/80 [==============================] - 0s 1ms/step - loss: 0.5396 - accuracy: 0.8425
Epoch 164/200
80/80 [==============================] - 0s 1ms/step - loss: 0.5033 - accuracy: 0.8550
Epoch 165/200
80/80 [==============================] - 0s 988us/step - loss: 0.5117 - accuracy: 0.8625
Epoch 166/200
80/80 [==============================] - 0s 963us/step - loss: 0.4719 - accuracy: 0.8750
Epoch 167/200
80/80 [==============================] - 0s 1ms/step - loss: 0.4827 - accuracy: 0.8450
Epoch 168/200
80/80 [==============================] - 0s 1ms/step - loss: 0.4848 - accuracy: 0.8650
Epoch 169/200
80/80 [==============================] - 0s 975us/step - loss: 0.4485 - accuracy: 0.8675
Epoch 170/200
80/80 [==============================] - 0s 1ms/step - loss: 0.5028 - accuracy: 0.8550
Epoch 171/200
80/80 [==============================] - 0s 1ms/step - loss: 0.4926 - accuracy: 0.852

In [15]:
model.evaluate(x_test_new, y_test)

4/4 [==============================] - 0s 1000us/step - loss: 0.5147 - accuracy: 0.8500


[0.5147266983985901, 0.8500000238418579]

In [16]:
# EXERCISE: Use the tf.saved_model API to save your model in the SavedModel format. 
export_dir = 'saved_model/1'

tf.saved_model.save(model, export_dir)

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_model/1\assets


In [17]:
# Select mode of optimization
mode = "Speed" 

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT

In [18]:
# EXERCISE: Use the TFLiteConverter SavedModel API to initialize the converter

converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)

# Set the optimzations
converter.optimizations = [optimization]

# Invoke the converter to finally generate the TFLite model
tflite_model = tflite_model = converter.convert()

In [21]:
tflite_model_file = pathlib.Path('./model.tflite')
tflite_model_file.write_bytes(tflite_model)

8960

In [22]:
model.save("model.h5")
print("Saved model to disk")

Saved model to disk


In [23]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                848       
_________________________________________________________________
batch_normalization_2 (Batch (None, 16)                64        
_________________________________________________________________
dropout_2 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 8)                 136       
_________________________________________________________________
batch_normalization_3 (Batch (None, 8)                 32        
_________________________________________________________________
dropout_3 (Dropout)          (None, 8)                 0         
_________________________________________________________________
dense_6 (Dense)              (None, 8)                

In [29]:
data = [
    [1,4,2,4,2,5,1,4,5,5,4,3,4,4,4,5,5,5,5,4,3,3,3,3,4,4,3,3,4,3,3,3,4,3,4,3,4,3,4,3,3,4,1,2,3,2,3,4,3,2,2,1]
]

In [30]:
X_pred = pd.DataFrame(data, index=["Abu"], columns=X.columns)
X_pred

,AE1,AE2,AE3,AE4,AE5,AE6,AE7,AE8,AE9,AE10,...,PE1,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10
Abu,1,4,2,4,2,5,1,4,5,5,...,1,2,3,2,3,4,3,2,2,1


In [31]:
model.predict(X_pred)

array([[4.7221780e-02, 9.5276660e-01, 1.1589783e-05]], dtype=float32)

In [35]:
print("Prediction: {}".format(tf.argmax(predictions, axis=1)))

Prediction: [1]


In [40]:
class_names = ['Authoration', 'Authorative', 'Permissive']

In [56]:
predict_dataset = tf.convert_to_tensor([
    [1,5,3,1,1,3,1,5,5,3,5,3,0,5,1,5,5,5,5,1,3,3,1,5,1,5,1,4,4,4,4,4,4,2,4,3,4,2,4,4,4,2,4,4,2,4,3,4,2,4,4,4], #Authoration
    [5,1,5,1,4,2,5,1,5,1,2,5,3,4,2,1,1,2,1,1,1,5,1,4,1,5,1,4,4,4,4,4,4,3,4,3,4,4,4,3,3,4,5,4,3,4,3,4,4,4,3,3], #Authoration
    [2,4,3,0,2,3,2,5,5,2,4,3,3,3,4,5,3,3,5,3,3,3,4,3,2,4,5,4,3,3,3,3,4,3,1,3,4,4,4,4,3,3,4,4,3,4,3,4,4,4,4,3], #Permissive
    [3,1,4,2,3,2,2,3,5,4,5,1,5,1,3,4,3,3,2,4,5,5,2,4,1,5,1,5,5,4,4,4,1,3,2,4,4,4,4,4,4,4,3,1,3,2,4,4,4,4,4,4], #Authoration
    [1,5,2,5,1,4,1,4,1,5,4,3,5,2,4,3,2,2,4,2,1,3,2,3,2,2,5,2,5,2,4,2,5,2,5,2,4,2,3,2,3,2,1,4,3,1,4,1,2,4,1,1], #Authoration
    [1,4,2,4,2,5,1,4,5,5,4,3,4,4,4,5,5,5,5,4,3,3,3,3,4,4,3,3,4,3,3,3,4,3,4,3,4,3,4,3,3,4,1,2,3,2,3,4,3,2,2,1], #authorative
    [1,3,2,4,2,4,1,3,2,5,5,2,3,3,4,5,4,4,3,4,1,2,1,3,1,0,4,4,3,5,3,3,5,2,1,3,3,3,5,5,4,4,5,5,2,4,3,3,3,5,5,4], #permissive
    [3,1,4,5,3,1,4,1,2,1,4,1,5,2,3,1,5,4,3,3,3,5,2,5,1,3,1,4,4,3,2,4,3,2,4,4,4,2,4,4,5,4,4,3,2,4,4,4,2,4,4,5] #permissive
])

In [57]:
predictions = model(predict_dataset, training=False)

In [58]:
for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {} ({:4.1f}%)".format(i, name, 100*p))

Example 0 prediction: Authoration (50.3%)
Example 1 prediction: Authoration (56.0%)
Example 2 prediction: Authoration (46.9%)
Example 3 prediction: Authoration (57.5%)
Example 4 prediction: Authoration (56.7%)
Example 5 prediction: Authorative (55.9%)
Example 6 prediction: Permissive (49.9%)
Example 7 prediction: Permissive (41.5%)


In [59]:
for i, logits in enumerate(predictions):
  class_idx = tf.argmax(logits).numpy()
  p = tf.nn.softmax(logits)[class_idx]
  name = class_names[class_idx]
  print("Example {} prediction: {}".format(i, name, 100*p))

Example 0 prediction: Authoration
Example 1 prediction: Authoration
Example 2 prediction: Authoration
Example 3 prediction: Authoration
Example 4 prediction: Authoration
Example 5 prediction: Authorative
Example 6 prediction: Permissive
Example 7 prediction: Permissive


In [60]:
class_idx

2